In [1]:
!wget https://launchpad.net/~mario-mariomedina/+archive/ubuntu/talib/+files/libta-lib0_0.4.0-oneiric1_amd64.deb -qO libta.deb
!wget https://launchpad.net/~mario-mariomedina/+archive/ubuntu/talib/+files/ta-lib0-dev_0.4.0-oneiric1_amd64.deb -qO ta.deb
!dpkg -i libta.deb ta.deb
!pip install ta-lib
import talib
!pip install mpl_finance
# from google.colab import drive
# drive.mount('/content/drive')

Selecting previously unselected package libta-lib0.
(Reading database ... 145480 files and directories currently installed.)
Preparing to unpack libta.deb ...
Unpacking libta-lib0 (0.4.0-oneiric1) ...
Selecting previously unselected package ta-lib0-dev.
Preparing to unpack ta.deb ...
Unpacking ta-lib0-dev (0.4.0-oneiric1) ...
Setting up libta-lib0 (0.4.0-oneiric1) ...
Setting up ta-lib0-dev (0.4.0-oneiric1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Processing triggers for libc-bin (2.27-3ubuntu1.2) ...
/sbin/ldconfig.real: /usr/local/lib/python3.6/dist-packages/ideep4py/lib/libmkldnn.so.0 is not a symbolic link

     |████████████████████████████████| 276kB 5.3MB/s 
  Created wheel for ta-lib: filename=TA_Lib-0.4.19-cp36-cp36m-linux_x86_64.whl size=1437796 sha256=46cd2f6038c326337ae71b602efece3cc5e27d605526275970da30577444e114
  Stored in directory: /root/.cache/pip/wheels/a3/f6/12/3d1ccd06caadd8fa47e016991dd0d27f1163bb260f1854e2ff
Successfully built ta-lib


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pandas_datareader as pdr
import seaborn as sns
import datetime as datetime
from sklearn import preprocessing
import requests
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense
from PIL import Image, ImageDraw, ImageFont
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
import csv
import statistics
import math
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SpatialDropout2D, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Conv1D,MaxPooling1D
from tensorflow.keras.layers import Activation,Dense ,Dropout ,GRU, ConvLSTM2D ,LSTM ,Bidirectional,TimeDistributed,Flatten, Conv1D, MaxPooling1D,BatchNormalization
from tensorflow.keras.optimizers import SGD, RMSprop
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import layers 
from tensorflow.keras.layers import LayerNormalization
from tensorflow.python.keras.layers import Input, Embedding, Dot, Reshape, Dense
from tensorflow.python.keras.models import Model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import Adamax, Adam
from tensorflow.keras.layers import Input,Embedding,LSTM,Dense
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
# gpu environment
import tensorflow as tf
from tensorflow.keras import *
gpus = tf.config.list_physical_devices("GPU")
if gpus:
    gpu0 = gpus[0] #如果有多个GPU，仅使用第0个GPU
    tf.config.experimental.set_memory_growth(gpu0, True) #设置GPU显存用量按需使用
    # 或者也可以设置GPU显存为固定使用量(例如：4G)
    # tf.config.experimental.set_virtual_device_configuration(gpu0,
    # [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)]) 
    tf.config.set_visible_devices([gpu0],"GPU") 

# useful functions
def exponential_smoothing(days, alpha, numdata):
  # assign weights
  weights = []
  i = days 
  while i>0:
    weights.append( math.exp(-alpha*i) )
    i-=1
  weights = np.array(weights) / sum(weights)
  # return data after smoothing
  results = []
  results += numdata[:days]
  for i in range(days, len(numdata)):
    results.append((np.dot( np.array(weights), np.array(numdata[i-days:i]))))
  return results

classes =['0', '1']
def to_onelist(text):
    onehot = [0] * 2
    onehot[ int(classes.index(str(text)) ) ] = 1
    label_list = np.array(onehot)
    return label_list

def convertData(df, record):
    x = []
    df.rename(columns={'Unnamed: 0':'Date'}, inplace=True)
    df = df.drop(['Date'], axis=1)
    df = df.drop(['MarginPurchaseLimit'], axis=1)
    df = df.drop(['ShortSaleLimit'], axis=1)
    df = df.dropna(axis=0, how='any')
    df.reset_index(drop=True, inplace=True)
    days_before = 30
    for i in range(days_before, len(df)):
        tmp = df[i-days_before:i]
        for col in df.columns:
          if col == 'Date' or col == 'y_label' or col =='bs_ratio' or col =='b_s_ratio': continue
          mean = record[str(col)+'_mean'][i-30]
          stdev = record[str(col)+'_stdev'][i-30]
          def test(s): 
            return (float(s)-mean)/stdev
          tmp[str(col)].map(test)
        x.append( np.array(np.array(tmp).tolist()[0: days_before]).reshape(days_before,len(tmp.columns)))
    y = list(df['y_label'][days_before:])
    # convert to one-hot encoding
    y_res = [] 
    for i in range(len(y)):
      y_res.append(to_onelist(str(y[i])))
    return x, y_res

def count_label_dist(arr):
    count_1 = 0 
    count_0 = 0 
    for i in range(len(arr)):
        if list(arr[i])== list([0,1]): count_1 += 1
        elif list(arr[i])== list([1,0]): count_0 += 1
    print(count_0, count_1)
    return count_0, count_1

# stocknum
stockNum = ["1101", "1102", "1216", "1301", "1326", "1402", "2002", "2105", "2207",
            "2301", "2303", "2308", "2317", "2327", "2330", "2352", "2357", "2382",
            "2395", "2408", "2412", "2454", "2474", "2609", "2610", "2801", "2823",
            "2880", "2881", "2882", "2883", "2884", "2885", "2886", "2887", "2888",
            "2890", "2891", "2892", "2912", "3008", "3045", "4904", "4938", "5871", "5880", "6505", "9904", "9910"]

Define our powerful model

In [3]:
def lstm4_model(input_shape , n_classes):
    model = Sequential()
    model.add(BatchNormalization(input_shape=(input_shape[0],input_shape[1])))
    model.add(LSTM(25,return_sequences=True))
    model.add(BatchNormalization())
    model.add(LSTM(12,return_sequences=True))
    model.add(BatchNormalization())
    model.add(LSTM(6,return_sequences=True))
    model.add(BatchNormalization())
    model.add(LSTM(3,return_sequences=True))
    model.add(Flatten())
    model.add(BatchNormalization())
    model.add(Dense(90, activation='relu', kernel_regularizer=keras.regularizers.l2(l=0.001)))
    model.add(BatchNormalization())
    model.add(Dense(25, activation='relu', kernel_regularizer=keras.regularizers.l2(l=0.001)))
    model.add(BatchNormalization())
    model.add(Dense(2, activation='sigmoid', kernel_regularizer=keras.regularizers.l2(l=0.001)))
    lr_schedule = keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=1e-4, decay_steps=100000, decay_rate=0.9)
    model.compile(loss="binary_crossentropy", optimizer=Adam(learning_rate=lr_schedule), metrics=['accuracy'])
    model.summary()
    return model

In [4]:
model = lstm4_model((30, 23), 2)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 30, 23)            92        
_________________________________________________________________
lstm (LSTM)                  (None, 30, 25)            4900      
_________________________________________________________________
batch_normalization_1 (Batch (None, 30, 25)            100       
_________________________________________________________________
lstm_1 (LSTM)                (None, 30, 12)            1824      
_________________________________________________________________
batch_normalization_2 (Batch (None, 30, 12)            48        
_________________________________________________________________
lstm_2 (LSTM)                (None, 30, 6)             456       
_________________________________________________________________
batch_normalization_3 (Batch (None, 30, 6)             2

Splitting Training, Validation, and Testing Data

In [ ]:
pd.options.mode.chained_assignment = None 

train_x = []
test_x = []
val_x = []
val_y = []
train_y = []
test_y = []

for stockno in stockNum:
# for stockno in ['1101']:
    df = pd.read_csv('/content/drive/My Drive/Program_Trading_Code/0050_stock_5_5/'+str(stockno)+'.csv')
    record = pd.read_csv('/content/drive/My Drive/Program_Trading_Code/0050_stock_5_5/'+str(stockno)+'mean&stdev.csv')
    x, y = convertData(df, record)
    #train:val:test = 8:1:1
    tr_va = int(len(x)*8/10)
    va_te = int(len(x)*9/10)
    train_x = train_x + x[:tr_va]
    train_y = train_y + y[:tr_va]
    val_x = val_x + x[tr_va: va_te]
    val_y = val_y + y[tr_va: va_te]
    test_x = test_x + x[va_te:]
    test_y = test_y +y[va_te:]
    print(str(stockno)+" loaded....")
    print(len(train_x), len(val_x), len(test_x))

train_x = np.array(train_x).reshape(len(train_x), 30, 23)
test_x  = np.array(test_x).reshape(len(test_x), 30, 23)
val_x   = np.array(val_x).reshape(len(val_x), 30, 23)
train_y = np.array(train_y).reshape(len(train_y),2)
test_y  = np.array(test_y).reshape(len(test_y),2)
val_y   = np.array(val_y).reshape(len(val_y),2)

Start Training Process

In [ ]:
for iteration in range(1,6):
    %load_ext tensorboard
    logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
    tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
    model = lstm4_model((30, 23), 2)
    callback = EarlyStopping(monitor="val_loss", patience=25, verbose=1, mode="auto")
    history = model.fit(train_x, train_y , epochs=50, batch_size= 256, validation_data=(val_x, val_y), callbacks=[callback, tensorboard_callback])
    # history = model.fit(train_x, train_y , epochs=150, batch_size= 256, validation_data=(val_x, val_y), callbacks=[tensorboard_callback])
    model.save('/content/drive/My Drive/Program_Trading_Code/LSTM4*5_Ensemble/lstm_ensemble_' + str(iteration) + '.h5')
    print(str(iteration) + "training done")

Start Testing

In [ ]:
# load in models
model1 = keras.models.load_model('/content/drive/My Drive/Program_Trading_Code/LSTM4*5_Ensemble/lstm_ensemble_1.h5')
model2 = keras.models.load_model('/content/drive/My Drive/Program_Trading_Code/LSTM4*5_Ensemble/lstm_ensemble_2.h5')
model3 = keras.models.load_model('/content/drive/My Drive/Program_Trading_Code/LSTM4*5_Ensemble/lstm_ensemble_3.h5')
model4 = keras.models.load_model('/content/drive/My Drive/Program_Trading_Code/LSTM4*5_Ensemble/lstm_ensemble_4.h5')
model5 = keras.models.load_model('/content/drive/My Drive/Program_Trading_Code/LSTM4*5_Ensemble/lstm_ensemble_5.h5')
# start testing and calculate accuracy
truth_y = []
for i in range(len(test_y)):
  truth_y.append(int(classes[np.argmax(test_y[i])]))
prediction_y = [0]*len(test_y)
for i in range(len(test_y)):
  prediction_y[i] += int(classes[np.argmax(model1.predict(test_x[i].reshape(1,30,23)))])
  prediction_y[i] += int(classes[np.argmax(model2.predict(test_x[i].reshape(1,30,23)))])
  prediction_y[i] += int(classes[np.argmax(model3.predict(test_x[i].reshape(1,30,23)))])
  prediction_y[i] += int(classes[np.argmax(model4.predict(test_x[i].reshape(1,30,23)))])
  prediction_y[i] += int(classes[np.argmax(model5.predict(test_x[i].reshape(1,30,23)))])
  if prediction_y[i] >= 3 : prediction_y[i] = 1
  else: prediction_y[i] = 0
print("Accuracy: ")
print( 1 - (np.sum(np.logical_xor(np.array(prediction_y), np.array(truth_y))))/len(test_y) )
# Accuracy: 
# 0.8850644735506142

Start calculating the Rate of Return Of Validation Set

In [ ]:
# Validation: start = '2018/08/01', end = '2019/06/03'
# Testing: start = '2019/06/04', end = '2020/11/11'
# stockNum
stockNum = ["1101", "1102", "1216", "1301", "1326", "1402", "2002", "2105", "2207",
            "2301", "2303", "2308", "2317", "2327", "2330", "2352", "2357", "2382",
            "2395", "2408", "2412", "2454", "2474", "2609", "2610", "2801", "2823",
            "2880", "2881", "2882", "2883", "2884", "2885", "2886", "2887", "2888",
            "2890", "2891", "2892", "2912", "3008", "3045", "4904", "4938", "5871", "5880", "6505", "9904", "9910"]

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('max_colwidth',100)
# Set the information of data
start = '2018/08/01'
end = '2019/06/03'
s = start.split('/')
e = end.split('/')
start = datetime.datetime(int(s[0]), int(s[1]), int(s[2]))
end = datetime.datetime(int(e[0]), int(e[1]), int(e[2]))

asset_at_the_start = 49 * 10**6
total_asset_present_value = 0
ROR_Validation = {}
for stock_num in stockNum:
  print(stock_num)
  Price_df = pdr.DataReader(str(stock_num)+'.TW', 'yahoo', start=start, end=end)
  # Return_Test_Stock_df = Price_df.copy(deep=True)
  Price_df.index = Price_df.index.format(formatter=lambda x: x.strftime('%Y-%m-%d'))
  stock_time = str(Price_df.index[0]).split(" ")[0]
  url = "https://api.finmindtrade.com/api/v3/data" #load in 融資融券
  parameter = {
    "dataset": "TaiwanStockMarginPurchaseShortSale",
    "stock_id": str(stock_num),
    "date": stock_time,
  }
  data = requests.get(url, params=parameter)
  data = data.json()
  data = pd.DataFrame(data['data'])
  data['b_total'] = data['MarginPurchaseLimit'] - data['MarginPurchaseTodayBalance']
  data['s_total'] = data['ShortSaleLimit'] - data['ShortSaleTodayBalance']
  data['b_s_ratio'] = (data['ShortSaleLimit'] - data['ShortSaleTodayBalance'])/(data['MarginPurchaseLimit'] - data['MarginPurchaseTodayBalance']+1)
  data = data.set_index('date')
  # 合併這兩個表格
  new_df = Price_df.join(data)
  new_df = new_df.drop(['stock_id'], axis=1)
  new_df = new_df.drop(['ShortSaleCashRepayment'], axis=1)
  url = "https://api.finmindtrade.com/api/v3/data" #load in三大法人投資情況
  parameter = {
    "dataset": "InstitutionalInvestorsBuySell",
    "stock_id": str(stock_num),
    "date": stock_time,
  }
  data = requests.get(url, params=parameter)
  data = data.json()
  data = pd.DataFrame(data['data'])
  df = pd.DataFrame()
  z = 0 
  i = 0
  while i < len(data)  :
    a = data['date'][i]
    count = 0
    sell = 0
    buy = 0 
    if i+6 < len(data)-1:
        for j  in range(i, i+6):
            if data['date'][j]== a :
              count += 1
              sell += data['sell'][j]
              buy += data['buy'][j]
    else:
      for j in range(i, len(data)):
        if data['date'][j] == a :
          count += 1
          sell += data['sell'][j]
          buy += data['buy'][j]
    df = df.append({'date': a, 'buy_volume': buy, 'sell_volume':sell, 'bs_ratio':buy/(sell+1)}, ignore_index=True)
    i+=count
  new_df = new_df.join(df.set_index('date'))
  new_df = new_df.fillna(axis=0,method='ffill')
  new_df = new_df.drop(['Note'], axis=1)
  new_df.reset_index(drop=True, inplace=True)
  for columnName in new_df.columns:
    if str(columnName) == 'y_label' or str(columnName) == 'index': continue
    new_df[str(columnName)] = exponential_smoothing(5, 0.2, list(new_df[str(columnName)]))
    #days = 10 # need to be 5
    days = 5
    ans = [0]*days
    for i in range(days,len(new_df)):
      Pi = new_df['Close'][i]
      Pj = new_df['Close'][i-days]
      if Pi > Pj : y = 1
      else: y = 0
      ans.append(y)
  new_df['y_label']= ans
  new_df = new_df.reset_index()
  new_df.rename(columns={'index':'Date'}, inplace=True)
  time_list = list(new_df['Date'])
  new_df = new_df.drop(['Date'], axis=1)
  new_df = new_df.drop(['MarginPurchaseLimit'], axis=1)
  new_df = new_df.drop(['ShortSaleLimit'], axis=1)
  Price_df = Price_df.reset_index()
  new_df.to_csv('/content/drive/My Drive/Program_Trading_Code/LSTM4*5_Ensemble/Validation/' + str(stock_num) + 'new_df.csv')
  Price_df.to_csv('/content/drive/My Drive/Program_Trading_Code/LSTM4*5_Ensemble/Validation/' + str(stock_num) + 'price_df.csv')

In [ ]:
# # Validation: start = '2018/08/01', end = '2019/06/03'
# # Testing: start = '2019/06/04', end = '2020/11/11'
# # load in models
from keras.models import load_model
model1 = keras.models.load_model('/content/drive/My Drive/Program_Trading_Code/LSTM4*5_Ensemble/lstm_ensemble_1.h5')
model2 = keras.models.load_model('/content/drive/My Drive/Program_Trading_Code/LSTM4*5_Ensemble/lstm_ensemble_2.h5')
model3 = keras.models.load_model('/content/drive/My Drive/Program_Trading_Code/LSTM4*5_Ensemble/lstm_ensemble_3.h5')
model4 = keras.models.load_model('/content/drive/My Drive/Program_Trading_Code/LSTM4*5_Ensemble/lstm_ensemble_4.h5')
model5 = keras.models.load_model('/content/drive/My Drive/Program_Trading_Code/LSTM4*5_Ensemble/lstm_ensemble_5.h5')
# stockNum
stockNum = ["1101", "1102", "1216", "1301", "1326", "1402", "2002", "2105", "2207",
            "2301", "2303", "2308", "2317", "2327", "2330", "2352", "2357", "2382",
            "2395", "2408", "2412", "2454", "2474", "2609", "2610", "2801", "2823",
            "2880", "2881", "2882", "2883", "2884", "2885", "2886", "2887", "2888",
            "2890", "2891", "2892", "2912", "3008", "3045", "4904", "4938", "5871", "5880", "6505", "9904", "9910"]

In [ ]:
# # # Validation: start = '2018/08/01', end = '2019/06/03'
# # # Testing: start = '2019/06/04', end = '2020/11/11'
# # # load in models
# from keras.models import load_model
# model1 = keras.models.load_model('/content/drive/My Drive/Program_Trading_Code/LSTM4*5_Ensemble/lstm_ensemble_1.h5')
# model2 = keras.models.load_model('/content/drive/My Drive/Program_Trading_Code/LSTM4*5_Ensemble/lstm_ensemble_2.h5')
# model3 = keras.models.load_model('/content/drive/My Drive/Program_Trading_Code/LSTM4*5_Ensemble/lstm_ensemble_3.h5')
# model4 = keras.models.load_model('/content/drive/My Drive/Program_Trading_Code/LSTM4*5_Ensemble/lstm_ensemble_4.h5')
# model5 = keras.models.load_model('/content/drive/My Drive/Program_Trading_Code/LSTM4*5_Ensemble/lstm_ensemble_5.h5')
# # stockNum
# stockNum = ["1101", "1102", "1216", "1301", "1326", "1402", "2002", "2105", "2207",
#             "2301", "2303", "2308", "2317", "2327", "2330", "2352", "2357", "2382",
#             "2395", "2408", "2412", "2454", "2474", "2609", "2610", "2801", "2823",
#             "2880", "2881", "2882", "2883", "2884", "2885", "2886", "2887", "2888",
#             "2890", "2891", "2892", "2912", "3008", "3045", "4904", "4938", "5871", "5880", "6505", "9904", "9910"]

asset_at_the_start = 49 * 10**6 #起始總資產
total_asset_present_value = 0 #現在手上握有的資產：股票加上現金
ROR_Validation = {} #驗證資料的投資報酬率hash table
for stock_num in stockNum:
#for stock_num in ["2207"]:
  print(stock_num)
  # read data in
  new_df = pd.read_csv('/content/drive/My Drive/Program_Trading_Code/LSTM4*5_Ensemble/Validation/' + str(stock_num) + 'new_df.csv')
  if 'Unnamed: 0' in new_df.columns: new_df = new_df.drop(['Unnamed: 0'], axis=1)
  Price_df = pd.read_csv('/content/drive/My Drive/Program_Trading_Code/LSTM4*5_Ensemble/Validation/' + str(stock_num) + 'price_df.csv')
  if 'Unnamed: 0' in Price_df.columns: Price_df = Price_df.drop(['Unnamed: 0'], axis=1)
  # variable declaration
  time_slide = 30 #模型吃資料的時間區間
  y_preds_list=[0]*(time_slide-1) #前面先塞29個〇，預測值都不看，從第三十天開始看起
  asset_for_this_stock = asset_at_the_start*(1/49) #起始時對於這個股票我可以動用的現金
  left_money = asset_for_this_stock #交易之後，我手頭上對於這支股票能操作的現金餘額
  own_stock = 0 #我擁有的這支股票總股數
  #own_asset = left_money
  asset_record = [left_money]*time_slide #記錄當前時間點剩下的現金餘額
  
  for i in range(time_slide, len(new_df)+1): #先對於所有資料算出預測出來的漲跌
    tmp = new_df[i-time_slide:i] # [0:30](0~29), [1:31](1~30), ..., [173,203](173~202), [174,204](174~203)
    for col in new_df.columns:
      if col == 'Date' or col == 'y_label' or col =='bs_ratio' or col =='b_s_ratio': continue
      mean = statistics.mean(tmp[str(col)])
      stdev = statistics.stdev(tmp[str(col)])
      if stdev == 0: stdev = 1
      def test(s): 
        return (float(s)-mean)/stdev
      tmp[str(col)].map(test)
    x = np.array(np.array(tmp).tolist()[0: time_slide]).reshape(time_slide,len(tmp.columns))
    predict = int(classes[np.argmax(model1.predict(x.reshape(1,30,23)))]) + int(classes[np.argmax(model2.predict(x.reshape(1,30,23)))]) + int(classes[np.argmax(model3.predict(x.reshape(1,30,23)))]) + int(classes[np.argmax(model4.predict(x.reshape(1,30,23)))]) + int(classes[np.argmax(model5.predict(x.reshape(1,30,23)))])
    predict = 1 if predict >= 3 else 0 #ensemble
    y_preds_list.append(predict)
  print(len(new_df), len(y_preds_list), len(new_df) == len(y_preds_list))

  stock_amount = [0]*(time_slide-1) #前面先塞29個〇，都不入場
  for i in range(time_slide, len(new_df)):
    price = int(Price_df['Close'][i]*100)/100
    if y_preds_list[i] == 1: #如果我現在要進場買入
      if y_preds_list[i-5] == 1: #如果我現在要買入 而且 五天前我也已經買入 --> 不出場被扣手續費，選擇補差額addition_money
        # 我可以補的差額 = {(對於這支股票我當前握有的資產總額/5)減去五天前的股數現在的價值, 我手頭上可以用的錢} 取小值，且如果小於零就不能再買
        addition_money = min( ((left_money+own_stock*price)/5) - stock_amount[i-5]*price, left_money)
      else: #如果我現在要買入 但 五天前我沒有買入 --> 直接看手頭上可以用多少錢
        # 我可以買的金額 = {(對於這支股票我當前握有的資產總額/5), 我手頭上可以用的錢} 取小值，且如果小於零就不能再買
        addition_money = min( (left_money+own_stock*price)/5, left_money)
      addition_money = 0 if addition_money < 0.0 else addition_money
      own_stock += int( addition_money / ( price * (1 + 0.001425)) ) #更新我本支股票累計的股數
      left_money -= int( addition_money / ( price * (1 + 0.001425)) )*price*(1 + 0.001425) #更新我對於本支股票手頭上還能動用的錢
      stock_amount.append( stock_amount[i-5] + int(addition_money/(price*(1+0.001425))) ) #更新我本支股票當天的股數
      #print("買","股票進場數: ",stock_amount[-1],"現金: ",left_money ,"資產：",left_money+own_stock*price)
    else: #如果我現在沒有要進場
      if y_preds_list[i-5] == 1: #如果我現在沒有要進場 而且 五天前我已經有買入 --> 則賣出
        own_stock -= stock_amount[i-5] #更新我本支股票累計的股數
        left_money += stock_amount[i-5]*price*(1 - 0.001425 - 0.003) #更新我對於本支股票手頭上還能動用的錢
        #print("賣","現金: ",left_money ,"資產：",left_money+own_stock*price)
      #如果我現在沒有要進場 而且 五天前我沒有買入 --> 沒有買賣交易的問題，單純更先今日入場股數為零即可
      stock_amount.append(0) #更新我本支股票當天的股數
    asset_record.append(left_money+own_stock*price) #更新我本支股票當前的總資產
  ROR_Validation[stock_num] = (asset_record[-1]-asset_for_this_stock)/asset_for_this_stock #記錄投資報酬率xx(xx*100％)
  print(ROR_Validation[stock_num])
  total_asset_present_value += asset_record[-1] #更新我累積所有股票的現值
print("ROR in Total = ", (total_asset_present_value-asset_at_the_start)/ asset_at_the_start)
print(ROR_Validation)

In [ ]:
ROR_Validation = {'1101': 0.0048301071304995575, '1102': 0.06996560157474935, '1216': -0.003416830709999777, '1301': -0.02507669572800014, '1326': -0.08949670953750018, '1402': -0.029615136123250244, '2002': -0.05509275493574993, '2105': -0.2291479477002502, '2207': 0.7921936477125, '2301': 0.1437060764392497, '2303': -0.13917272617924994, '2308': -0.03558072291250062, '2317': -0.0545405800940001, '2327': -0.2357191788, '2330': -0.06471114068749978, '2352': -0.004555931830750081, '2357': -0.16752275866249985, '2382': 0.05459429875099974, '2395': 0.1312711646617501, '2408': -0.07218358562450042, '2412': -0.04098252848750039, '2454': 0.04679071308750019, '2474': -0.23715112948750008, '2609': -0.07165964653675015, '2610': -0.03286127661350009, '2801': 0.1033390372324997, '2823': -0.1371243316432502, '2880': 0.07188242202425016, '2881': -0.08859919802724955, '2882': -0.1338354106495001, '2883': -0.07705958711874998, '2884': 0.2722334095092501, '2885': 0.11148684184499959, '2886': 0.12584401370174939, '2887': -0.030309146704000076, '2888': -0.10170561613500002, '2890': 0.16163566886874978, '2891': -0.04338470604174969, '2892': 0.07844732249400009, '2912': -0.1982071514125001, '3008': 0.05717320412500006, '3045': 0.021107189874999693, '4904': 0.010172696450249876, '4938': -0.08515053526225, '5871': 0.05659118860249955, '5880': 0.04798937971224974, '6505': -0.02949161421250005, '9904': 0.0985889143697495, '9910': -0.03808232151924994}
# define the dictionary of validation rate of return per year
stockNum = ["1101", "1102", "1216", "1301", "1326", "1402", "2002", "2105", "2207",
            "2301", "2303", "2308", "2317", "2327", "2330", "2352", "2357", "2382",
            "2395", "2408", "2412", "2454", "2474", "2609", "2610", "2801", "2823",
            "2880", "2881", "2882", "2883", "2884", "2885", "2886", "2887", "2888",
            "2890", "2891", "2892", "2912", "3008", "3045", "4904", "4938", "5871", "5880", "6505", "9904", "9910"]
# make the dictionary of validation asset allocation per year
Weights_Dict = {}
for stock_num in stockNum:
  if ROR_Validation[stock_num] > 0: Weights_Dict[stock_num] = ROR_Validation[stock_num]
  else: Weights_Dict[stock_num] = 0
WeightSum = sum(Weights_Dict.values())
for stock_num in stockNum:
  Weights_Dict[stock_num] /= WeightSum
  
print(sum(Weights_Dict.values()))
print(Weights_Dict)

1.0
{'1101': 0.0019635835825522973, '1102': 0.028443117902721134, '1216': 0.0, '1301': 0.0, '1326': 0.0, '1402': 0.0, '2002': 0.0, '2105': 0.0, '2207': 0.3220505050556923, '2301': 0.05842083514614129, '2303': 0.0, '2308': 0.0, '2317': 0.0, '2327': 0.0, '2330': 0.0, '2352': 0.0, '2357': 0.0, '2382': 0.022194221749555935, '2395': 0.053365670124520106, '2408': 0.0, '2412': 0.0, '2454': 0.019021829858466885, '2474': 0.0, '2609': 0.0, '2610': 0.0, '2801': 0.04201042160435691, '2823': 0.0, '2880': 0.029222362971960632, '2881': 0.0, '2882': 0.0, '2883': 0.0, '2884': 0.11067105533936955, '2885': 0.04532274883410388, '2886': 0.05115937029779387, '2887': 0.0, '2888': 0.0, '2890': 0.06570975284200595, '2891': 0.0, '2892': 0.031891192137693355, '2912': 0.0, '3008': 0.02324262422108024, '3045': 0.008580706471427045, '4904': 0.00413550656337777, '4938': 0.0, '5871': 0.023006017434958212, '5880': 0.01950912383388398, '6505': 0.0, '9904': 0.040079354028338586, '9910': 0.0}


Compute the testing rate of return 計算測試資料的投資報酬率

In [ ]:
# Validation: start = '2018/08/01', end = '2019/06/03'
# Testing: start = '2019/06/04', end = '2020/09/21'
# load in models
from keras.models import load_model
model1 = keras.models.load_model('/content/drive/My Drive/Program_Trading_Code/LSTM4*5_Ensemble/lstm_ensemble_1.h5')
model2 = keras.models.load_model('/content/drive/My Drive/Program_Trading_Code/LSTM4*5_Ensemble/lstm_ensemble_2.h5')
model3 = keras.models.load_model('/content/drive/My Drive/Program_Trading_Code/LSTM4*5_Ensemble/lstm_ensemble_3.h5')
model4 = keras.models.load_model('/content/drive/My Drive/Program_Trading_Code/LSTM4*5_Ensemble/lstm_ensemble_4.h5')
model5 = keras.models.load_model('/content/drive/My Drive/Program_Trading_Code/LSTM4*5_Ensemble/lstm_ensemble_5.h5')
# stockNum
stockNum = ["1101", "1102", "1216", "1301", "1326", "1402", "2002", "2105", "2207",
            "2301", "2303", "2308", "2317", "2327", "2330", "2352", "2357", "2382",
            "2395", "2408", "2412", "2454", "2474", "2609", "2610", "2801", "2823",
            "2880", "2881", "2882", "2883", "2884", "2885", "2886", "2887", "2888",
            "2890", "2891", "2892", "2912", "3008", "3045", "4904", "4938", "5871", "5880", "6505", "9904", "9910"]

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('max_colwidth',100)
# Set the information of data
start = '2019/06/04'
end = '2020/09/21'
s = start.split('/')
e = end.split('/')
start = datetime.datetime(int(s[0]), int(s[1]), int(s[2]))
end = datetime.datetime(int(e[0]), int(e[1]), int(e[2]))

asset_at_the_start = 20 * 10**6
total_asset_present_value = 0
ROR_Test = {}
for stock_num in stockNum:
  #if Weights_Dict[stock_num] == 0.0 : continue
  #先算出之後回測用的價位
  Price_df = pdr.DataReader(str(stock_num)+'.TW', 'yahoo', start=start, end=end)
  Return_Test_Stock_df = Price_df.copy(deep=True)
  Price_df.index = Price_df.index.format(formatter=lambda x: x.strftime('%Y-%m-%d'))
  stock_time = str(Price_df.index[0]).split(" ")[0]
  url = "https://api.finmindtrade.com/api/v3/data" #load in 融資融券
  parameter = {
    "dataset": "TaiwanStockMarginPurchaseShortSale",
    "stock_id": str(stock_num),
    "date": stock_time,
  }
  data = requests.get(url, params=parameter)
  data = data.json()
  data = pd.DataFrame(data['data'])
  data['b_total'] = data['MarginPurchaseLimit'] - data['MarginPurchaseTodayBalance']
  data['s_total'] = data['ShortSaleLimit'] - data['ShortSaleTodayBalance']
  data['b_s_ratio'] = (data['ShortSaleLimit'] - data['ShortSaleTodayBalance'])/(data['MarginPurchaseLimit'] - data['MarginPurchaseTodayBalance']+1)
  data = data.set_index('date')
  # 合併這兩個表格
  new_df = Price_df.join(data)
  new_df = new_df.drop(['stock_id'], axis=1)
  new_df = new_df.drop(['ShortSaleCashRepayment'], axis=1)
  url = "https://api.finmindtrade.com/api/v3/data" #load in三大法人投資情況
  parameter = {
    "dataset": "InstitutionalInvestorsBuySell",
    "stock_id": str(stock_num),
    "date": stock_time,
  }
  data = requests.get(url, params=parameter)
  data = data.json()
  data = pd.DataFrame(data['data'])
  df = pd.DataFrame()
  z = 0 
  i = 0
  while i < len(data)  :
    a = data['date'][i]
    count = 0
    sell = 0
    buy = 0 
    if i+6 < len(data)-1:
        for j  in range(i, i+6):
            if data['date'][j]== a :
              count += 1
              sell += data['sell'][j]
              buy += data['buy'][j]
    else:
      for j in range(i, len(data)):
        if data['date'][j] == a :
          count += 1
          sell += data['sell'][j]
          buy += data['buy'][j]
    df = df.append({'date': a, 'buy_volume': buy, 'sell_volume':sell, 'bs_ratio':buy/(sell+1)}, ignore_index=True)
    i+=count
  new_df = new_df.join(df.set_index('date'))
  new_df = new_df.fillna(axis=0,method='ffill')
  new_df = new_df.drop(['Note'], axis=1)
  new_df.reset_index(drop=True, inplace=True)
  for columnName in new_df.columns:
    if str(columnName) == 'y_label' or str(columnName) == 'index': continue
    new_df[str(columnName)] = exponential_smoothing(5, 0.2, list(new_df[str(columnName)]))
    days = 5
    ans = [0]*days
    for i in range(days,len(new_df)):
      Pi = new_df['Close'][i]
      Pj = new_df['Close'][i-days]
      if Pi > Pj : y = 1
      else: y = 0
      ans.append(y)
  new_df['y_label']= ans
  new_df = new_df.reset_index()
  new_df.rename(columns={'index':'Date'}, inplace=True)
  time_list = list(new_df['Date'])
  new_df = new_df.drop(['Date'], axis=1)
  new_df = new_df.drop(['MarginPurchaseLimit'], axis=1)
  new_df = new_df.drop(['ShortSaleLimit'], axis=1)
  Price_df = Price_df.reset_index()
  new_df.to_csv('/content/drive/My Drive/Program_Trading_Code/LSTM4*5_Ensemble/Testing0921/' + str(stock_num) + 'new_df.csv')
  Price_df.to_csv('/content/drive/My Drive/Program_Trading_Code/LSTM4*5_Ensemble/Testing0921/' + str(stock_num) + 'price_df.csv')

將＄切割成五份，30進35出

In [ ]:
Weights_Dict = {'1101': 0.0019635835825522973, '1102': 0.028443117902721134, '1216': 0.0, '1301': 0.0, '1326': 0.0, '1402': 0.0, '2002': 0.0, '2105': 0.0, '2207': 0.3220505050556923, '2301': 0.05842083514614129, '2303': 0.0, '2308': 0.0, '2317': 0.0, '2327': 0.0, '2330': 0.0, '2352': 0.0, '2357': 0.0, '2382': 0.022194221749555935, '2395': 0.053365670124520106, '2408': 0.0, '2412': 0.0, '2454': 0.019021829858466885, '2474': 0.0, '2609': 0.0, '2610': 0.0, '2801': 0.04201042160435691, '2823': 0.0, '2880': 0.029222362971960632, '2881': 0.0, '2882': 0.0, '2883': 0.0, '2884': 0.11067105533936955, '2885': 0.04532274883410388, '2886': 0.05115937029779387, '2887': 0.0, '2888': 0.0, '2890': 0.06570975284200595, '2891': 0.0, '2892': 0.031891192137693355, '2912': 0.0, '3008': 0.02324262422108024, '3045': 0.008580706471427045, '4904': 0.00413550656337777, '4938': 0.0, '5871': 0.023006017434958212, '5880': 0.01950912383388398, '6505': 0.0, '9904': 0.040079354028338586, '9910': 0.0}
# load in models
from keras.models import load_model
model1 = keras.models.load_model('/content/drive/My Drive/Program_Trading_Code/LSTM4*5_Ensemble/lstm_ensemble_1.h5')
model2 = keras.models.load_model('/content/drive/My Drive/Program_Trading_Code/LSTM4*5_Ensemble/lstm_ensemble_2.h5')
model3 = keras.models.load_model('/content/drive/My Drive/Program_Trading_Code/LSTM4*5_Ensemble/lstm_ensemble_3.h5')
model4 = keras.models.load_model('/content/drive/My Drive/Program_Trading_Code/LSTM4*5_Ensemble/lstm_ensemble_4.h5')
model5 = keras.models.load_model('/content/drive/My Drive/Program_Trading_Code/LSTM4*5_Ensemble/lstm_ensemble_5.h5')
# stockNum
stockNum = ["1101", "1102", "1216", "1301", "1326", "1402", "2002", "2105", "2207",
            "2301", "2303", "2308", "2317", "2327", "2330", "2352", "2357", "2382",
            "2395", "2408", "2412", "2454", "2474", "2609", "2610", "2801", "2823",
            "2880", "2881", "2882", "2883", "2884", "2885", "2886", "2887", "2888",
            "2890", "2891", "2892", "2912", "3008", "3045", "4904", "4938", "5871", "5880", "6505", "9904", "9910"]

asset_at_the_start = 20 * 10**6 #起始總資產
total_asset_present_value = 0 #現在手上握有的資產：股票加上現金
ROR_Test = {} #驗證資料的投資報酬率hash table
for stock_num in stockNum:
#for stock_num in ["1101"]:
  if Weights_Dict[stock_num] == 0.0 : continue
  print(stock_num)
  # read data in
  new_df = pd.read_csv('/content/drive/My Drive/Program_Trading_Code/LSTM4*5_Ensemble/Testing0921/' + str(stock_num) + 'new_df.csv')
  if 'Unnamed: 0' in new_df.columns: new_df = new_df.drop(['Unnamed: 0'], axis=1)
  Price_df = pd.read_csv('/content/drive/My Drive/Program_Trading_Code/LSTM4*5_Ensemble/Testing0921/' + str(stock_num) + 'price_df.csv')
  if 'Unnamed: 0' in Price_df.columns: Price_df = Price_df.drop(['Unnamed: 0'], axis=1)
  # variable declaration
  time_slide = 30 #模型吃資料的時間區間
  y_preds_list=[0]*(time_slide-1) #前面先塞29個〇，預測值都不看，從第三十天開始看起
  asset_for_this_stock = int(asset_at_the_start*Weights_Dict[stock_num]) #起始時對於這個股票我可以動用的現金
  left_money = asset_for_this_stock #交易之後，我手頭上對於這支股票能操作的現金餘額
  own_stock = 0 #我擁有的這支股票總股數
  #own_asset = left_money
  asset_record = [left_money]*time_slide #記錄當前時間點剩下的現金餘額
  print("asset_for_this_stock: ", asset_for_this_stock)
  for i in range(time_slide, len(new_df)+1): #先對於所有資料算出預測出來的漲跌
    tmp = new_df[i-time_slide:i] # [0:30](0~29), [1:31](1~30), ..., [173,203](173~202), [174,204](174~203)
    for col in new_df.columns:
      if col == 'Date' or col == 'y_label' or col =='bs_ratio' or col =='b_s_ratio': continue
      mean = statistics.mean(tmp[str(col)])
      stdev = statistics.stdev(tmp[str(col)])
      if stdev == 0: stdev = 1
      def test(s): 
        return (float(s)-mean)/stdev
      tmp[str(col)].map(test)
    x = np.array(np.array(tmp).tolist()[0: time_slide]).reshape(time_slide,len(tmp.columns))
    predict = int(classes[np.argmax(model1.predict(x.reshape(1,30,23)))]) + int(classes[np.argmax(model2.predict(x.reshape(1,30,23)))]) + int(classes[np.argmax(model3.predict(x.reshape(1,30,23)))]) + int(classes[np.argmax(model4.predict(x.reshape(1,30,23)))]) + int(classes[np.argmax(model5.predict(x.reshape(1,30,23)))])
    predict = 1 if predict >= 3 else 0 #ensemble
    y_preds_list.append(predict)
  #print(len(new_df), len(y_preds_list), len(new_df) == len(y_preds_list))

  stock_amount = [0]*(time_slide-1) #前面先塞29個〇，都不入場
  for i in range(time_slide, len(new_df)):
    price = int(Price_df['Close'][i]*100)/100
    if y_preds_list[i] == 1: #如果我現在要進場買入
      if y_preds_list[i-5] == 1 and stock_amount[i-5] != 0: #如果我現在要買入 而且 五天前我也已經買入 --> 不出場被扣手續費，選擇補差額addition_money
        # 我可以補的差額 = {(對於這支股票我當前握有的資產總額/5)減去五天前的股數現在的價值, 我手頭上可以用的錢} 取小值，且如果小於零就不能再買
        addition_money = min( ((left_money+own_stock*price)/5) - stock_amount[i-5]*price, left_money)
      else: #如果我現在要買入 但 五天前我沒有買入 --> 直接看手頭上可以用多少錢
        # 我可以買的金額 = {(對於這支股票我當前握有的資產總額/5), 我手頭上可以用的錢} 取小值，且如果小於零就不能再買
        addition_money = min( (left_money+own_stock*price)/5, left_money)
      addition_money = 0 if addition_money < 0.0 else addition_money
      own_stock += int( addition_money / ( price * (1 + 0.001425)) ) #更新我本支股票累計的股數
      left_money -= int( addition_money / ( price * (1 + 0.001425)) )*price*(1 + 0.001425) #更新我對於本支股票手頭上還能動用的錢
      stock_amount.append( stock_amount[i-5] + int(addition_money/(price*(1+0.001425))) ) #更新我本支股票當天的股數
      print("買", "股票進場數: ",stock_amount[-1], "持有總股數：", own_stock, "手上可用的現金: ",left_money ,"這支股票的總資產：",left_money+own_stock*price)
    else: #如果我現在沒有要進場
      if y_preds_list[i-5] == 1 and stock_amount[i-5] != 0: #如果我現在沒有要進場 而且 五天前我已經有買入 --> 則賣出
        own_stock -= stock_amount[i-5] #更新我本支股票累計的股數
        left_money += stock_amount[i-5]*price*(1 - 0.001425 - 0.003) #更新我對於本支股票手頭上還能動用的錢
        print("賣", "持有總股數：", own_stock, "手上可用的現金: ",left_money ,"資產：",left_money+own_stock*price)
      #如果我現在沒有要進場 而且 五天前我沒有買入 --> 沒有買賣交易的問題，單純更先今日入場股數為零即可
      stock_amount.append(0) #更新我本支股票當天的股數
    asset_record.append(left_money+own_stock*price) #更新我本支股票當前的總資產
  ROR_Test[stock_num] = (asset_record[-1]-asset_for_this_stock)/asset_for_this_stock #記錄投資報酬率xx(xx*100％)
  print(ROR_Test[stock_num])
  total_asset_present_value += asset_record[-1] #更新我累積所有股票的現值
print("ROR_Test in Total = ", (total_asset_present_value-asset_at_the_start)/ asset_at_the_start)
print(ROR_Test)

In [ ]:
ROR_Test in Total =  0.18525198772212473
{'1101': 0.0685205874054642, '1102': -0.04865320873471982, '2207': 0.6201950749533068, '2301': -0.13688934463388894, '2382': -0.022481598802277895, '2395': 0.10938510189208732, '2454': 0.8830980345248609, '2801': -0.10809113131361574, '2880': -0.05775613852025978, '2884': 0.08203221830844609, '2885': -0.03238878383238441, '2886': -0.022257865395084283, '2890': -0.13771258825345561, '2892': -0.02381053635804989, '3008': -0.2156674659353954, '3045': -0.08947004430145543, '4904': -0.16413659257647217, '5871': -0.08854641295314275, '5880': -0.010441238556622434, '9904': -0.22721585112969647}

將＄切割成五份，30進35出，但中間有跌就出場



In [ ]:
# Validation: start = '2018/08/01', end = '2019/06/03'
# Testing: start = '2019/06/04', end = '2020/11/11'
# load in models
from keras.models import load_model
model1 = keras.models.load_model('/content/drive/My Drive/Program_Trading_Code/LSTM4*5_Ensemble/lstm_ensemble_1.h5')
model2 = keras.models.load_model('/content/drive/My Drive/Program_Trading_Code/LSTM4*5_Ensemble/lstm_ensemble_2.h5')
model3 = keras.models.load_model('/content/drive/My Drive/Program_Trading_Code/LSTM4*5_Ensemble/lstm_ensemble_3.h5')
model4 = keras.models.load_model('/content/drive/My Drive/Program_Trading_Code/LSTM4*5_Ensemble/lstm_ensemble_4.h5')
model5 = keras.models.load_model('/content/drive/My Drive/Program_Trading_Code/LSTM4*5_Ensemble/lstm_ensemble_5.h5')
# stockNum
stockNum = ["1101", "1102", "1216", "1301", "1326", "1402", "2002", "2105", "2207",
            "2301", "2303", "2308", "2317", "2327", "2330", "2352", "2357", "2382",
            "2395", "2408", "2412", "2454", "2474", "2609", "2610", "2801", "2823",
            "2880", "2881", "2882", "2883", "2884", "2885", "2886", "2887", "2888",
            "2890", "2891", "2892", "2912", "3008", "3045", "4904", "4938", "5871", "5880", "6505", "9904", "9910"]

asset_at_the_start = 49 * 10**6 #起始總資產
total_asset_present_value = 0 #現在手上握有的資產：股票加上現金
ROR_Validation = {} #驗證資料的投資報酬率hash table
for stock_num in stockNum:
  print(stock_num)
  # read data in
  new_df = pd.read_csv('/content/drive/My Drive/Program_Trading_Code/LSTM4*5_Ensemble/Validation/' + str(stock_num) + 'new_df.csv')
  if 'Unnamed: 0' in new_df.columns: new_df = new_df.drop(['Unnamed: 0'], axis=1)
  Price_df = pd.read_csv('/content/drive/My Drive/Program_Trading_Code/LSTM4*5_Ensemble/Validation/' + str(stock_num) + 'price_df.csv')
  if 'Unnamed: 0' in Price_df.columns: Price_df = Price_df.drop(['Unnamed: 0'], axis=1)
  # variable declaration
  time_slide = 30 #模型吃資料的時間區間
  y_preds_list=[0]*(time_slide-1) #前面先塞29個〇，預測值都不看，從第三十天開始看起
  asset_for_this_stock = asset_at_the_start*(1/49) #起始時對於這個股票我可以動用的現金
  left_money = asset_for_this_stock #交易之後，我手頭上對於這支股票能操作的現金餘額
  own_stock = 0 #我擁有的這支股票總股數
  asset_record = [left_money]*time_slide #記錄當前時間點剩下的現金餘額
  
  for i in range(time_slide, len(new_df)+1): #先對於所有資料算出預測出來的漲跌
    tmp = new_df[i-time_slide:i] # [0:30](0~29), [1:31](1~30), ..., [173,203](173~202), [174,204](174~203)
    for col in new_df.columns:
      if col == 'Date' or col == 'y_label' or col =='bs_ratio' or col =='b_s_ratio': continue
      mean = statistics.mean(tmp[str(col)])
      stdev = statistics.stdev(tmp[str(col)])
      if stdev == 0: stdev = 1
      def test(s): 
        return (float(s)-mean)/stdev
      tmp[str(col)].map(test)
    x = np.array(np.array(tmp).tolist()[0: time_slide]).reshape(time_slide,len(tmp.columns))
    predict = int(classes[np.argmax(model1.predict(x.reshape(1,30,23)))]) + int(classes[np.argmax(model2.predict(x.reshape(1,30,23)))]) + int(classes[np.argmax(model3.predict(x.reshape(1,30,23)))]) + int(classes[np.argmax(model4.predict(x.reshape(1,30,23)))]) + int(classes[np.argmax(model5.predict(x.reshape(1,30,23)))])
    predict = 1 if predict >= 3 else 0 #ensemble
    y_preds_list.append(predict)
  print(len(new_df), len(y_preds_list), len(new_df) == len(y_preds_list))

  stock_amount = [0]*(time_slide-1) #前面先塞29個〇，都不入場
  for i in range(time_slide-1, len(new_df)):
    price = int(Price_df['Close'][i]*100)/100
    #看i-4~i-1之間，如果其預測是漲而且有持股，要檢查他是否已經跌價，如果是就立馬出場
    for j in range(i-4, i):
      if y_preds_list[j] == 1 and stock_amount[j] != 0 and price < (int(Price_df['Close'][j]*100)/100) : #如果今日價格小於他入場日的價格，就馬上出場
        own_stock -= stock_amount[j] #更新我本支股票累計的股數
        left_money += stock_amount[j]*price*(1 - 0.001425 - 0.003) #更新我對於本支股票手頭上還能動用的錢
        stock_amount[j] = 0 #更新j天的股票數量為0
        # print("提早出場", "持有總股數：", own_stock, "手上可用的現金: ",left_money ,"資產：",left_money+own_stock*price)
    #預測5天後漲則今天進場，五天前如果有買則今天離場（30進35出）
    if y_preds_list[i] == 1: #如果我現在要進場買入
      if y_preds_list[i-5] == 1 and stock_amount[i-5] != 0: #如果我現在要買入 而且 五天前我也已經買入 --> 不出場被扣手續費，選擇補差額addition_money
        # 我可以補的差額 = {(對於這支股票我當前握有的資產總額/5)減去五天前的股數現在的價值, 我手頭上可以用的錢} 取小值，且如果小於零就不能再買
        addition_money = min( ((left_money+own_stock*price)/5) - stock_amount[i-5]*price, left_money)
      else: #如果我現在要買入 但 五天前我沒有買入 --> 直接看手頭上可以用多少錢
        # 我可以買的金額 = {(對於這支股票我當前握有的資產總額/5), 我手頭上可以用的錢} 取小值，且如果小於零就不能再買
        addition_money = min( (left_money+own_stock*price)/5, left_money)
      addition_money = 0 if addition_money < 0.0 else addition_money
      own_stock += int( addition_money / ( price * (1 + 0.001425)) ) #更新我本支股票累計的股數
      left_money -= int( addition_money / ( price * (1 + 0.001425)) )*price*(1 + 0.001425) #更新我對於本支股票手頭上還能動用的錢
      stock_amount.append( stock_amount[i-5] + int(addition_money/(price*(1+0.001425))) ) #更新我本支股票當天的股數
      #print("買","股票進場數: ",stock_amount[-1],"現金: ",left_money ,"資產：",left_money+own_stock*price)
    else: #如果我現在沒有要進場
      if y_preds_list[i-5] == 1 and stock_amount[i-5] != 0: #如果我現在沒有要進場 而且 五天前我已經有買入 --> 則賣出
        own_stock -= stock_amount[i-5] #更新我本支股票累計的股數
        left_money += stock_amount[i-5]*price*(1 - 0.001425 - 0.003) #更新我對於本支股票手頭上還能動用的錢
        #print("賣","現金: ",left_money ,"資產：",left_money+own_stock*price)
      #如果我現在沒有要進場 而且 五天前我沒有買入 --> 沒有買賣交易的問題，單純更先今日入場股數為零即可
      stock_amount.append(0) #更新我本支股票當天的股數
    asset_record.append(left_money+own_stock*price) #更新我本支股票當前的總資產
  ROR_Validation[stock_num] = (asset_record[-1]-asset_for_this_stock)/asset_for_this_stock #記錄投資報酬率xx(xx*100％)
  print(ROR_Validation[stock_num])
  total_asset_present_value += asset_record[-1] #更新我累積所有股票的現值
print("ROR in Total = ", (total_asset_present_value-asset_at_the_start)/ asset_at_the_start)
print(ROR_Validation)

In [ ]:
# ROR_Validation 
# define the dictionary of validation rate of return per year
stockNum = ["1101", "1102", "1216", "1301", "1326", "1402", "2002", "2105", "2207",
            "2301", "2303", "2308", "2317", "2327", "2330", "2352", "2357", "2382",
            "2395", "2408", "2412", "2454", "2474", "2609", "2610", "2801", "2823",
            "2880", "2881", "2882", "2883", "2884", "2885", "2886", "2887", "2888",
            "2890", "2891", "2892", "2912", "3008", "3045", "4904", "4938", "5871", "5880", "6505", "9904", "9910"]
# make the dictionary of validation asset allocation per year
Weights_Dict = {}
for stock_num in stockNum:
  if ROR_Validation[stock_num] > 0: Weights_Dict[stock_num] = ROR_Validation[stock_num]
  else: Weights_Dict[stock_num] = 0
WeightSum = sum(Weights_Dict.values())
for stock_num in stockNum:
  Weights_Dict[stock_num] /= WeightSum
  
print(sum(Weights_Dict.values()))
print(Weights_Dict)

In [ ]:
Weights_Dict = {'1101': 0.0, '1102': 0.0, '1216': 0.0, '1301': 0.0, '1326': 0.0, '1402': 0.0, '2002': 0.0, '2105': 0.0, '2207': 0.43079208734712354, '2301': 0.04930409559958762, '2303': 0.0, '2308': 0.0, '2317': 0.07690629791720573, '2327': 0.0, '2330': 0.0, '2352': 0.0, '2357': 0.0, '2382': 0.0, '2395': 0.14500719004000273, '2408': 0.0, '2412': 0.0, '2454': 0.0, '2474': 0.0, '2609': 0.0, '2610': 0.0, '2801': 0.0, '2823': 0.0, '2880': 0.0, '2881': 0.0, '2882': 0.0, '2883': 0.0, '2884': 0.2848404627652492, '2885': 0.0, '2886': 0.0, '2887': 0.0, '2888': 0.0, '2890': 0.0, '2891': 0.0, '2892': 0.0, '2912': 0.0, '3008': 0.0, '3045': 0.0, '4904': 0.0, '4938': 0.0, '5871': 0.0, '5880': 0.013149866330830977, '6505': 0.0, '9904': 0.0, '9910': 0.0}
# load in models
from keras.models import load_model
model1 = keras.models.load_model('/content/drive/My Drive/Program_Trading_Code/LSTM4*5_Ensemble/lstm_ensemble_1.h5')
model2 = keras.models.load_model('/content/drive/My Drive/Program_Trading_Code/LSTM4*5_Ensemble/lstm_ensemble_2.h5')
model3 = keras.models.load_model('/content/drive/My Drive/Program_Trading_Code/LSTM4*5_Ensemble/lstm_ensemble_3.h5')
model4 = keras.models.load_model('/content/drive/My Drive/Program_Trading_Code/LSTM4*5_Ensemble/lstm_ensemble_4.h5')
model5 = keras.models.load_model('/content/drive/My Drive/Program_Trading_Code/LSTM4*5_Ensemble/lstm_ensemble_5.h5')
# stockNum
stockNum = ["1101", "1102", "1216", "1301", "1326", "1402", "2002", "2105", "2207",
            "2301", "2303", "2308", "2317", "2327", "2330", "2352", "2357", "2382",
            "2395", "2408", "2412", "2454", "2474", "2609", "2610", "2801", "2823",
            "2880", "2881", "2882", "2883", "2884", "2885", "2886", "2887", "2888",
            "2890", "2891", "2892", "2912", "3008", "3045", "4904", "4938", "5871", "5880", "6505", "9904", "9910"]

asset_at_the_start = 20 * 10**6 #起始總資產
total_asset_present_value = 0 #現在手上握有的資產：股票加上現金
ROR_Test = {} #驗證資料的投資報酬率hash table
for stock_num in stockNum:
#for stock_num in ["1101"]:
  if Weights_Dict[stock_num] == 0.0 : continue
  print(stock_num)
  # read data in
  new_df = pd.read_csv('/content/drive/My Drive/Program_Trading_Code/LSTM4*5_Ensemble/Testing0921/' + str(stock_num) + 'new_df.csv')
  if 'Unnamed: 0' in new_df.columns: new_df = new_df.drop(['Unnamed: 0'], axis=1)
  Price_df = pd.read_csv('/content/drive/My Drive/Program_Trading_Code/LSTM4*5_Ensemble/Testing0921/' + str(stock_num) + 'price_df.csv')
  if 'Unnamed: 0' in Price_df.columns: Price_df = Price_df.drop(['Unnamed: 0'], axis=1)
  # variable declaration
  time_slide = 30 #模型吃資料的時間區間
  y_preds_list=[0]*(time_slide-1) #前面先塞29個〇，預測值都不看，從第三十天開始看起
  asset_for_this_stock = int(asset_at_the_start*Weights_Dict[stock_num]) #起始時對於這個股票我可以動用的現金
  left_money = asset_for_this_stock #交易之後，我手頭上對於這支股票能操作的現金餘額
  own_stock = 0 #我擁有的這支股票總股數
  #own_asset = left_money
  asset_record = [left_money]*time_slide #記錄當前時間點剩下的現金餘額
  print("asset_for_this_stock: ", asset_for_this_stock)
  for i in range(time_slide, len(new_df)+1): #先對於所有資料算出預測出來的漲跌
    tmp = new_df[i-time_slide:i] # [0:30](0~29), [1:31](1~30), ..., [173,203](173~202), [174,204](174~203)
    for col in new_df.columns:
      if col == 'Date' or col == 'y_label' or col =='bs_ratio' or col =='b_s_ratio': continue
      mean = statistics.mean(tmp[str(col)])
      stdev = statistics.stdev(tmp[str(col)])
      if stdev == 0: stdev = 1
      def test(s): 
        return (float(s)-mean)/stdev
      tmp[str(col)].map(test)
    x = np.array(np.array(tmp).tolist()[0: time_slide]).reshape(time_slide,len(tmp.columns))
    predict = int(classes[np.argmax(model1.predict(x.reshape(1,30,23)))]) + int(classes[np.argmax(model2.predict(x.reshape(1,30,23)))]) + int(classes[np.argmax(model3.predict(x.reshape(1,30,23)))]) + int(classes[np.argmax(model4.predict(x.reshape(1,30,23)))]) + int(classes[np.argmax(model5.predict(x.reshape(1,30,23)))])
    predict = 1 if predict >= 3 else 0 #ensemble
    y_preds_list.append(predict)
  #print(len(new_df), len(y_preds_list), len(new_df) == len(y_preds_list))

  stock_amount = [0]*(time_slide-1) #前面先塞29個〇，都不入場
  for i in range(time_slide-1, len(new_df)):
    print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
    price = int(Price_df['Close'][i]*100)/100 #今日價格
    print("index: ",i,"現在價格：", price)
    #看i-4~i-1之間，如果其預測是漲而且有持股，要檢查他是否已經跌價，如果是就立馬出場
    for j in range(i-4, i):
      if y_preds_list[j] == 1 and stock_amount[j] != 0 and price > (int(Price_df['Close'][j]*100)/100)*1.04 : #如果今日價格小於他入場日的價格，就馬上出場 #更動：12月20
          own_stock -= stock_amount[j] #更新我本支股票累計的股數
          left_money += stock_amount[j]*price*(1 - 0.001425 - 0.003) #更新我對於本支股票手頭上還能動用的錢
          stock_amount[j] = 0 #更新j天的股票數量為0
          print("賺到提早出場", "持有總股數：", own_stock, "手上可用的現金: ",left_money ,"資產：",left_money+own_stock*price)
      #更動12月20
      if y_preds_list[j] == 1 and stock_amount[j] != 0 and price <(int(Price_df['Close'][j]*100)/100)*1.04 : #如果今日價格小於他入場日的價格，就馬上出場 #更動：12月20
        addition_money = min( ((left_money+own_stock*price)/5) , left_money)
        addition_money = 0 if addition_money < 0.0 else addition_money
        own_stock += int( addition_money / ( price * (1 + 0.001425)) ) #更新我本支股票累計的股數
        left_money -= int( addition_money / ( price * (1 + 0.001425)) )*price*(1 + 0.001425) #更新我對於本支股票手頭上還能動用的錢
        stock_amount[j]+= int( addition_money / ( price * (1 + 0.001425)) ) #
        print("虧錢加碼", "持有總股數：", own_stock, "手上可用的現金: ",left_money ,"資產：",left_money+own_stock*price)
    #預測5天後漲則今天進場，五天前如果有買則今天離場（30進35出）
    if y_preds_list[i] == 1: #如果我現在要進場買入
      #if y_preds_list[i-5] == 1: 
      if y_preds_list[i-5] == 1 and stock_amount[i-5] != 0: #如果我現在要買入 而且 五天前我也已經買入 --> 不出場被扣手續費，選擇補差額addition_money
        # 我可以補的差額 = {(對於這支股票我當前握有的資產總額/5)減去五天前的股數現在的價值, 我手頭上可以用的錢} 取小值，且如果小於零就不能再買
        addition_money = min( ((left_money+own_stock*price)/5) - stock_amount[i-5]*price, left_money)
      else: #如果我現在要買入 但 五天前我沒有買入 --> 直接看手頭上可以用多少錢
        # 我可以買的金額 = {(對於這支股票我當前握有的資產總額/5), 我手頭上可以用的錢} 取小值，且如果小於零就不能再買
        addition_money = min( (left_money+own_stock*price)/5, left_money)
      addition_money = 0 if addition_money < 0.0 else addition_money
      own_stock += int( addition_money / ( price * (1 + 0.001425)) ) #更新我本支股票累計的股數
      left_money -= int( addition_money / ( price * (1 + 0.001425)) )*price*(1 + 0.001425) #更新我對於本支股票手頭上還能動用的錢
      stock_amount.append( stock_amount[i-5] + int(addition_money/(price*(1+0.001425))) ) #更新我本支股票當天的股數
      print("買", "股票進場數: ",stock_amount[-1], "持有總股數：", own_stock, "手上可用的現金: ",left_money ,"這支股票的總資產：",left_money+own_stock*price)
    else: #如果我現在沒有要進場
      #if y_preds_list[i-5] == 1: 
      if y_preds_list[i-5] == 1 and stock_amount[i-5] != 0: #如果我現在沒有要進場 而且 五天前我已經有買入 --> 則賣出
        own_stock -= stock_amount[i-5] #更新我本支股票累計的股數
        left_money += stock_amount[i-5]*price*(1 - 0.001425 - 0.003) #更新我對於本支股票手頭上還能動用的錢
        print("賣", "持有總股數：", own_stock, "手上可用的現金: ",left_money ,"資產：",left_money+own_stock*price)
      #如果我現在沒有要進場 而且 五天前我沒有買入 --> 沒有買賣交易的問題，單純更先今日入場股數為零即可
      stock_amount.append(0) #更新我本支股票當天的股數
    asset_record.append(left_money+own_stock*price) #更新我本支股票當前的總資產
  ROR_Test[stock_num] = (asset_record[-1]-asset_for_this_stock)/asset_for_this_stock #記錄投資報酬率xx(xx*100％)
  print(ROR_Test[stock_num])
  total_asset_present_value += asset_record[-1] #更新我累積所有股票的現值
print("ROR_Test in Total = ", (total_asset_present_value-asset_at_the_start)/ asset_at_the_start)
print(ROR_Test)

將＄切割成五份，30進35出，但中間有跌就進場，有漲就出場

In [ ]:
# Validation: start = '2018/08/01', end = '2019/06/03'
# Testing: start = '2019/06/04', end = '2020/11/11'
# load in models
from keras.models import load_model
model1 = keras.models.load_model('/content/drive/My Drive/Program_Trading_Code/LSTM4*5_Ensemble/lstm_ensemble_1.h5')
model2 = keras.models.load_model('/content/drive/My Drive/Program_Trading_Code/LSTM4*5_Ensemble/lstm_ensemble_2.h5')
model3 = keras.models.load_model('/content/drive/My Drive/Program_Trading_Code/LSTM4*5_Ensemble/lstm_ensemble_3.h5')
model4 = keras.models.load_model('/content/drive/My Drive/Program_Trading_Code/LSTM4*5_Ensemble/lstm_ensemble_4.h5')
model5 = keras.models.load_model('/content/drive/My Drive/Program_Trading_Code/LSTM4*5_Ensemble/lstm_ensemble_5.h5')
# stockNum
stockNum = ["1101", "1102", "1216", "1301", "1326", "1402", "2002", "2105", "2207",
            "2301", "2303", "2308", "2317", "2327", "2330", "2352", "2357", "2382",
            "2395", "2408", "2412", "2454", "2474", "2609", "2610", "2801", "2823",
            "2880", "2881", "2882", "2883", "2884", "2885", "2886", "2887", "2888",
            "2890", "2891", "2892", "2912", "3008", "3045", "4904", "4938", "5871", "5880", "6505", "9904", "9910"]

asset_at_the_start = 49 * 10**6 #起始總資產
total_asset_present_value = 0 #現在手上握有的資產：股票加上現金
ROR_Validation = {} #驗證資料的投資報酬率hash table
for stock_num in stockNum:
  print(stock_num)
  # read data in
  new_df = pd.read_csv('/content/drive/My Drive/Program_Trading_Code/LSTM4*5_Ensemble/Validation/' + str(stock_num) + 'new_df.csv')
  if 'Unnamed: 0' in new_df.columns: new_df = new_df.drop(['Unnamed: 0'], axis=1)
  Price_df = pd.read_csv('/content/drive/My Drive/Program_Trading_Code/LSTM4*5_Ensemble/Validation/' + str(stock_num) + 'price_df.csv')
  if 'Unnamed: 0' in Price_df.columns: Price_df = Price_df.drop(['Unnamed: 0'], axis=1)
  # variable declaration
  time_slide = 30 #模型吃資料的時間區間
  y_preds_list=[0]*(time_slide-1) #前面先塞29個〇，預測值都不看，從第三十天開始看起
  asset_for_this_stock = asset_at_the_start*(1/49) #起始時對於這個股票我可以動用的現金
  left_money = asset_for_this_stock #交易之後，我手頭上對於這支股票能操作的現金餘額
  own_stock = 0 #我擁有的這支股票總股數
  asset_record = [left_money]*time_slide #記錄當前時間點剩下的現金餘額
  
  for i in range(time_slide, len(new_df)+1): #先對於所有資料算出預測出來的漲跌
    tmp = new_df[i-time_slide:i] # [0:30](0~29), [1:31](1~30), ..., [173,203](173~202), [174,204](174~203)
    for col in new_df.columns:
      if col == 'Date' or col == 'y_label' or col =='bs_ratio' or col =='b_s_ratio': continue
      mean = statistics.mean(tmp[str(col)])
      stdev = statistics.stdev(tmp[str(col)])
      if stdev == 0: stdev = 1
      def test(s): 
        return (float(s)-mean)/stdev
      tmp[str(col)].map(test)
    x = np.array(np.array(tmp).tolist()[0: time_slide]).reshape(time_slide,len(tmp.columns))
    predict = int(classes[np.argmax(model1.predict(x.reshape(1,30,23)))]) + int(classes[np.argmax(model2.predict(x.reshape(1,30,23)))]) + int(classes[np.argmax(model3.predict(x.reshape(1,30,23)))]) + int(classes[np.argmax(model4.predict(x.reshape(1,30,23)))]) + int(classes[np.argmax(model5.predict(x.reshape(1,30,23)))])
    predict = 1 if predict >= 3 else 0 #ensemble
    y_preds_list.append(predict)
  print(len(new_df), len(y_preds_list), len(new_df) == len(y_preds_list))

  stock_amount = [0]*(time_slide-1) #前面先塞29個〇，都不入場
  for i in range(time_slide-1, len(new_df)):
    price = int(Price_df['Close'][i]*100)/100
    #看i-4~i-1之間，如果其預測是漲而且有持股，要檢查他是否已經跌價，如果是就立馬出場
    for j in range(i-4, i):
      if y_preds_list[j] == 1 and stock_amount[j] != 0 and price < (int(Price_df['Close'][j]*100)/100) : #如果今日價格小於他入場日的價格，就馬上加碼，因為exponential所以我預測漲
        addition_money = min( ((left_money+own_stock*price)/5) , left_money)
        addition_money = 0 if addition_money < 0.0 else addition_money
        own_stock += int( addition_money / ( price * (1 + 0.001425)) ) #更新我本支股票累計的股數
        left_money -= int( addition_money / ( price * (1 + 0.001425)) )*price*(1 + 0.001425) #更新我對於本支股票手頭上還能動用的錢
        stock_amount[j]+= int( addition_money / ( price * (1 + 0.001425)) )
        print("跌價加碼", "持有總股數：", own_stock, "手上可用的現金: ",left_money ,"資產：",left_money+own_stock*price)
    #預測5天後漲則今天進場，五天前如果有買則今天離場（30進35出）
    if y_preds_list[i] == 1: #如果我現在要進場買入
      if y_preds_list[i-5] == 1 and stock_amount[i-5] != 0: #如果我現在要買入 而且 五天前我也已經買入 --> 不出場被扣手續費，選擇補差額addition_money
        # 我可以補的差額 = {(對於這支股票我當前握有的資產總額/5)減去五天前的股數現在的價值, 我手頭上可以用的錢} 取小值，且如果小於零就不能再買
        addition_money = min( ((left_money+own_stock*price)/5) - stock_amount[i-5]*price, left_money)
      else: #如果我現在要買入 但 五天前我沒有買入 --> 直接看手頭上可以用多少錢
        # 我可以買的金額 = {(對於這支股票我當前握有的資產總額/5), 我手頭上可以用的錢} 取小值，且如果小於零就不能再買
        addition_money = min( (left_money+own_stock*price)/5, left_money)
      addition_money = 0 if addition_money < 0.0 else addition_money
      own_stock += int( addition_money / ( price * (1 + 0.001425)) ) #更新我本支股票累計的股數
      left_money -= int( addition_money / ( price * (1 + 0.001425)) )*price*(1 + 0.001425) #更新我對於本支股票手頭上還能動用的錢
      stock_amount.append( stock_amount[i-5] + int(addition_money/(price*(1+0.001425))) ) #更新我本支股票當天的股數
      #print("買","股票進場數: ",stock_amount[-1],"現金: ",left_money ,"資產：",left_money+own_stock*price)
    else: #如果我現在沒有要進場
      if y_preds_list[i-5] == 1 and stock_amount[i-5] != 0: #如果我現在沒有要進場 而且 五天前我已經有買入 --> 則賣出
        own_stock -= stock_amount[i-5] #更新我本支股票累計的股數
        left_money += stock_amount[i-5]*price*(1 - 0.001425 - 0.003) #更新我對於本支股票手頭上還能動用的錢
        #print("賣","現金: ",left_money ,"資產：",left_money+own_stock*price)
      #如果我現在沒有要進場 而且 五天前我沒有買入 --> 沒有買賣交易的問題，單純更先今日入場股數為零即可
      stock_amount.append(0) #更新我本支股票當天的股數
    asset_record.append(left_money+own_stock*price) #更新我本支股票當前的總資產
  ROR_Validation[stock_num] = (asset_record[-1]-asset_for_this_stock)/asset_for_this_stock #記錄投資報酬率xx(xx*100％)
  print(ROR_Validation[stock_num])
  total_asset_present_value += asset_record[-1] #更新我累積所有股票的現值
print("ROR in Total = ", (total_asset_present_value-asset_at_the_start)/ asset_at_the_start)
print(ROR_Validation)

In [ ]:
# ROR_Validation 
# define the dictionary of validation rate of return per year
stockNum = ["1101", "1102", "1216", "1301", "1326", "1402", "2002", "2105", "2207",
            "2301", "2303", "2308", "2317", "2327", "2330", "2352", "2357", "2382",
            "2395", "2408", "2412", "2454", "2474", "2609", "2610", "2801", "2823",
            "2880", "2881", "2882", "2883", "2884", "2885", "2886", "2887", "2888",
            "2890", "2891", "2892", "2912", "3008", "3045", "4904", "4938", "5871", "5880", "6505", "9904", "9910"]
# make the dictionary of validation asset allocation per year
Weights_Dict = {}
for stock_num in stockNum:
  if ROR_Validation[stock_num] > 0: Weights_Dict[stock_num] = ROR_Validation[stock_num]
  else: Weights_Dict[stock_num] = 0
WeightSum = sum(Weights_Dict.values())
for stock_num in stockNum:
  Weights_Dict[stock_num] /= WeightSum
  
print(sum(Weights_Dict.values()))
print(Weights_Dict)

1.0
{'1101': 0.0037291788286125138, '1102': 0.06398628839443782, '1216': 0.0, '1301': 0.0, '1326': 0.0, '1402': 0.0, '2002': 0.0, '2105': 0.0, '2207': 0.37492948913770124, '2301': 0.04593931908037911, '2303': 0.0, '2308': 0.0, '2317': 0.0, '2327': 0.0, '2330': 0.0, '2352': 0.0, '2357': 0.0, '2382': 0.019375507520422802, '2395': 0.03988901306451477, '2408': 0.0, '2412': 0.0, '2454': 0.0, '2474': 0.0, '2609': 0.0, '2610': 0.0, '2801': 0.0, '2823': 0.0, '2880': 0.034660184685436635, '2881': 0.0, '2882': 0.0, '2883': 0.0, '2884': 0.13433415485747077, '2885': 0.029438529336723616, '2886': 0.06490548427877942, '2887': 0.0, '2888': 0.0, '2890': 0.04881782625595282, '2891': 0.0, '2892': 0.032653189975113364, '2912': 0.0, '3008': 0.07375192007181906, '3045': 0.0, '4904': 0.0, '4938': 0.0, '5871': 0.0, '5880': 0.019502112430243815, '6505': 0.0, '9904': 0.014087802082392324, '9910': 0.0}


In [ ]:
Weights_Dict = {'1101': 0.0037291788286125138, '1102': 0.06398628839443782, '1216': 0.0, '1301': 0.0, '1326': 0.0, '1402': 0.0, '2002': 0.0, '2105': 0.0, '2207': 0.37492948913770124, '2301': 0.04593931908037911, '2303': 0.0, '2308': 0.0, '2317': 0.0, '2327': 0.0, '2330': 0.0, '2352': 0.0, '2357': 0.0, '2382': 0.019375507520422802, '2395': 0.03988901306451477, '2408': 0.0, '2412': 0.0, '2454': 0.0, '2474': 0.0, '2609': 0.0, '2610': 0.0, '2801': 0.0, '2823': 0.0, '2880': 0.034660184685436635, '2881': 0.0, '2882': 0.0, '2883': 0.0, '2884': 0.13433415485747077, '2885': 0.029438529336723616, '2886': 0.06490548427877942, '2887': 0.0, '2888': 0.0, '2890': 0.04881782625595282, '2891': 0.0, '2892': 0.032653189975113364, '2912': 0.0, '3008': 0.07375192007181906, '3045': 0.0, '4904': 0.0, '4938': 0.0, '5871': 0.0, '5880': 0.019502112430243815, '6505': 0.0, '9904': 0.014087802082392324, '9910': 0.0}
# load in models
from keras.models import load_model
model1 = keras.models.load_model('/content/drive/My Drive/Program_Trading_Code/LSTM4*5_Ensemble/lstm_ensemble_1.h5')
model2 = keras.models.load_model('/content/drive/My Drive/Program_Trading_Code/LSTM4*5_Ensemble/lstm_ensemble_2.h5')
model3 = keras.models.load_model('/content/drive/My Drive/Program_Trading_Code/LSTM4*5_Ensemble/lstm_ensemble_3.h5')
model4 = keras.models.load_model('/content/drive/My Drive/Program_Trading_Code/LSTM4*5_Ensemble/lstm_ensemble_4.h5')
model5 = keras.models.load_model('/content/drive/My Drive/Program_Trading_Code/LSTM4*5_Ensemble/lstm_ensemble_5.h5')
# stockNum
stockNum = ["1101", "1102", "1216", "1301", "1326", "1402", "2002", "2105", "2207",
            "2301", "2303", "2308", "2317", "2327", "2330", "2352", "2357", "2382",
            "2395", "2408", "2412", "2454", "2474", "2609", "2610", "2801", "2823",
            "2880", "2881", "2882", "2883", "2884", "2885", "2886", "2887", "2888",
            "2890", "2891", "2892", "2912", "3008", "3045", "4904", "4938", "5871", "5880", "6505", "9904", "9910"]

asset_at_the_start = 20 * 10**6 #起始總資產
total_asset_present_value = 0 #現在手上握有的資產：股票加上現金
ROR_Test = {} #驗證資料的投資報酬率hash table
for stock_num in stockNum:
#for stock_num in ["1101"]:
  if Weights_Dict[stock_num] == 0.0 : continue
  print(stock_num)
  # read data in
  new_df = pd.read_csv('/content/drive/My Drive/Program_Trading_Code/LSTM4*5_Ensemble/Testing0921/' + str(stock_num) + 'new_df.csv')
  if 'Unnamed: 0' in new_df.columns: new_df = new_df.drop(['Unnamed: 0'], axis=1)
  Price_df = pd.read_csv('/content/drive/My Drive/Program_Trading_Code/LSTM4*5_Ensemble/Testing0921/' + str(stock_num) + 'price_df.csv')
  if 'Unnamed: 0' in Price_df.columns: Price_df = Price_df.drop(['Unnamed: 0'], axis=1)
  # variable declaration
  time_slide = 30 #模型吃資料的時間區間
  y_preds_list=[0]*(time_slide-1) #前面先塞29個〇，預測值都不看，從第三十天開始看起
  asset_for_this_stock = int(asset_at_the_start*Weights_Dict[stock_num]) #起始時對於這個股票我可以動用的現金
  left_money = asset_for_this_stock #交易之後，我手頭上對於這支股票能操作的現金餘額
  own_stock = 0 #我擁有的這支股票總股數
  #own_asset = left_money
  asset_record = [left_money]*time_slide #記錄當前時間點剩下的現金餘額
  print("asset_for_this_stock: ", asset_for_this_stock)
  for i in range(time_slide, len(new_df)+1): #先對於所有資料算出預測出來的漲跌
    tmp = new_df[i-time_slide:i] # [0:30](0~29), [1:31](1~30), ..., [173,203](173~202), [174,204](174~203)
    for col in new_df.columns:
      if col == 'Date' or col == 'y_label' or col =='bs_ratio' or col =='b_s_ratio': continue
      mean = statistics.mean(tmp[str(col)])
      stdev = statistics.stdev(tmp[str(col)])
      if stdev == 0: stdev = 1
      def test(s): 
        return (float(s)-mean)/stdev
      tmp[str(col)].map(test)
    x = np.array(np.array(tmp).tolist()[0: time_slide]).reshape(time_slide,len(tmp.columns))
    predict = int(classes[np.argmax(model1.predict(x.reshape(1,30,23)))]) + int(classes[np.argmax(model2.predict(x.reshape(1,30,23)))]) + int(classes[np.argmax(model3.predict(x.reshape(1,30,23)))]) + int(classes[np.argmax(model4.predict(x.reshape(1,30,23)))]) + int(classes[np.argmax(model5.predict(x.reshape(1,30,23)))])
    predict = 1 if predict >= 3 else 0 #ensemble
    y_preds_list.append(predict)
  #print(len(new_df), len(y_preds_list), len(new_df) == len(y_preds_list))

  stock_amount = [0]*(time_slide-1) #前面先塞29個〇，都不入場
  for i in range(time_slide-1, len(new_df)):
    price = int(Price_df['Close'][i]*100)/100
    #看i-4~i-1之間，如果其預測是漲而且有持股，要檢查他是否已經跌價，如果是就立馬出場
    for j in range(i-4, i):
      if y_preds_list[j] == 1 and stock_amount[j] != 0 and price < (int(Price_df['Close'][j]*100)/100) : #如果今日價格小於他入場日的價格，就馬上加碼
        addition_money = min( ((left_money+own_stock*price)/5) , left_money)
        addition_money = 0 if addition_money < 0.0 else addition_money
        own_stock += int( addition_money / ( price * (1 + 0.001425)) ) #更新我本支股票累計的股數
        left_money -= int( addition_money / ( price * (1 + 0.001425)) )*price*(1 + 0.001425) #更新我對於本支股票手頭上還能動用的錢
        stock_amount[j]+= int( addition_money / ( price * (1 + 0.001425)) )
        print("跌價加碼", "Price today = ", str(price), "Previous Price = ", str((int(Price_df['Close'][j]*100)/100)), "持有總股數：", own_stock, "手上可用的現金: ",left_money ,"資產：",left_money+own_stock*price)
    #預測5天後漲則今天進場，五天前如果有買則今天離場（30進35出）
    if y_preds_list[i] == 1: #如果我現在要進場買入
      if y_preds_list[i-5] == 1 and stock_amount[i-5] != 0: #如果我現在要買入 而且 五天前我也已經買入 --> 不出場被扣手續費，選擇補差額addition_money
        # 我可以補的差額 = {(對於這支股票我當前握有的資產總額/5)減去五天前的股數現在的價值, 我手頭上可以用的錢} 取小值，且如果小於零就不能再買
        addition_money = min( ((left_money+own_stock*price)/5) - stock_amount[i-5]*price, left_money)
      else: #如果我現在要買入 但 五天前我沒有買入 --> 直接看手頭上可以用多少錢
        # 我可以買的金額 = {(對於這支股票我當前握有的資產總額/5), 我手頭上可以用的錢} 取小值，且如果小於零就不能再買
        addition_money = min( (left_money+own_stock*price)/5, left_money)
      addition_money = 0 if addition_money < 0.0 else addition_money
      own_stock += int( addition_money / ( price * (1 + 0.001425)) ) #更新我本支股票累計的股數
      left_money -= int( addition_money / ( price * (1 + 0.001425)) )*price*(1 + 0.001425) #更新我對於本支股票手頭上還能動用的錢
      stock_amount.append( stock_amount[i-5] + int(addition_money/(price*(1+0.001425))) ) #更新我本支股票當天的股數
      print("買", "Price today = ", str(price), "Previous Price = ", str((int(Price_df['Close'][i-5]*100)/100)), "股票進場數: ",stock_amount[-1], "持有總股數：", own_stock, "手上可用的現金: ",left_money ,"這支股票的總資產：",left_money+own_stock*price)
    else: #如果我現在沒有要進場
      if y_preds_list[i-5] == 1 and stock_amount[i-5] != 0: #如果我現在沒有要進場 而且 五天前我已經有買入 --> 則賣出
        own_stock -= stock_amount[i-5] #更新我本支股票累計的股數
        left_money += stock_amount[i-5]*price*(1 - 0.001425 - 0.003) #更新我對於本支股票手頭上還能動用的錢
        print("賣", "Price today = ", str(price), "Previous Price = ", str((int(Price_df['Close'][i-5]*100)/100)), "持有總股數：", own_stock, "手上可用的現金: ",left_money ,"資產：",left_money+own_stock*price)
      #如果我現在沒有要進場 而且 五天前我沒有買入 --> 沒有買賣交易的問題，單純更先今日入場股數為零即可
      stock_amount.append(0) #更新我本支股票當天的股數
    asset_record.append(left_money+own_stock*price) #更新我本支股票當前的總資產
  ROR_Test[stock_num] = (asset_record[-1]-asset_for_this_stock)/asset_for_this_stock #記錄投資報酬率xx(xx*100％)
  print(ROR_Test[stock_num])
  total_asset_present_value += asset_record[-1] #更新我累積所有股票的現值
print("ROR_Test in Total = ", (total_asset_present_value-asset_at_the_start)/ asset_at_the_start)
print(ROR_Test)

Streaming output truncated to the last 5000 lines.
0.6874606629793945
2301
asset_for_this_stock:  918786
買 Price today =  47.2 Previous Price =  46.04 股票進場數:  3887 持有總股數： 3887 手上可用的現金:  735058.16038 這支股票的總資產： 918524.56038
跌價加碼 Price today =  47.09 Previous Price =  47.2 持有總股數： 7780 手上可用的現金:  551475.55742775 資產： 917835.7574277499
買 Price today =  47.09 Previous Price =  46.7 股票進場數:  3892 持有總股數： 11672 手上可用的現金:  367940.11157874996 這支股票的總資產： 917574.59157875
跌價加碼 Price today =  47.09 Previous Price =  47.2 持有總股數： 15563 手上可用的現金:  184451.82283299995 資產： 917313.492833
買 Price today =  47.09 Previous Price =  46.95 股票進場數:  3890 持有總股數： 19453 手上可用的現金:  1010.6911904999579 這支股票的總資產： 917052.4611905
跌價加碼 Price today =  47.09 Previous Price =  47.2 持有總股數： 19474 手上可用的現金:  20.39202224995779 資產： 917051.05202225
買 Price today =  47.09 Previous Price =  46.79 股票進場數:  0 持有總股數： 19474 手上可用的現金:  20.39202224995779 這支股票的總資產： 917051.05202225
跌價加碼 Price today =  43.0 Previous Price =  47.2 持有總股數： 19474 手上可用的現金:  2

In [6]:
# stockNum
# stockNum = ["1101", "1102", "1216", "1301", "1326", "1402", "2002", "2105", "2207",
#             "2301", "2303", "2308", "2317", "2327", "2330", "2352", "2357", "2382",
#             "2395", "2408", "2412", "2454", "2474", "2609", "2610", "2801", "2823",
#             "2880", "2881", "2882", "2883", "2884", "2885", "2886", "2887", "2888",
#             "2890", "2891", "2892", "2912", "3008", "3045", "4904", "4938", "5871", "5880", "6505", "9904", "9910"]
stockNum = ['0050']
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('max_colwidth',100)
# Set the information of data
start = '2019/06/04'
end = '2020/09/21'
s = start.split('/')
e = end.split('/')
start = datetime.datetime(int(s[0]), int(s[1]), int(s[2]))
end = datetime.datetime(int(e[0]), int(e[1]), int(e[2]))

for stock_num in stockNum:
  print(stock_num)
  Price_df = pdr.DataReader(str(stock_num)+'.TW', 'yahoo', start=start, end=end)
  Price_df.index = Price_df.index.format(formatter=lambda x: x.strftime('%Y-%m-%d'))
  # stock_time = str(Price_df.index[0]).split(" ")[0]
  print(Price_df['Close'][0])
  print(Price_df['Close'][-1])
  print( ( Price_df['Close'][-1] - Price_df['Close'][0] ) / Price_df['Close'][0])

0050
78.05000305175781
103.0999984741211
0.3209480389866444
